In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
import haplotype
from haplotype.dataset import *
from haplotype.data_preprocess import *
from haplotype.utilities import *
from haplotype.predict_model import BEDICT_HaplotypeModel

In [3]:
curr_pth = os.path.abspath('..')
print(curr_pth)

/mnt/orisenbazuru/crispr


### Read sample data

In [4]:
df = pd.read_csv(os.path.join(curr_pth, 'sample_data', 'bystander_sampledata.csv'))
del df['Unnamed: 4']
df

,Editor,Inp_seq,Outp_seq,true_score
0,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGAATTTGTTGAGGGCGA,0.521655
1,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGAGTTTGTTGAGGGCGA,0.110202
2,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGGATTTGTTGAGGGCGA,0.180943
3,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGGGTTTGTTGAGGGCGA,0.168752
4,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGAATTTGTTGAGGGCGA,0.005775
5,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGAGTTTGTTGAGGGCGA,0.002406
6,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGGATTTGTTGAGGGCGA,0.001604
7,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGGGTTTGTTGAGGGCGA,0.004331
8,ABEmax,GAGAATGTCCCAGCCTGCCC,GAGAATGTCCCAGCCTGCCC,0.621697
9,ABEmax,GAGAATGTCCCAGCCTGCCC,GAGAGTGTCCCAGCCTGCCC,0.285483


The models expect sequences (i.e. target sites) to be wrapped in a `pandas.DataFrame` with a column representing the `ID` of a sequence (optional) and the `input sequence itself` (i.e. `protospacer sequence`). In the above example, **`Inp_seq`** represents the input sequence column.
The sequences should be of length 20 (i.e. 20 bases) and represent the protospacer target site.

In [5]:
# create a directory where we dump the predictions of the models
csv_dir = create_directory(os.path.join(curr_pth, 'sample_data', 'predictions_haplo'))

### Specify device (i.e. CPU or GPU) to run the models on

Specify device to run the model on. The models can run on `GPU` or `CPU`. We can instantiate a device by running `get_device(to_gpu,gpu_index)` function. 

- To run on GPU we pass `to_gpu = True` and specify which card to use if we have multiple cards `gpu_index=int` (i.e. in case we have multiple GPU cards we specify the index counting from 0). 
- If there is no GPU installed, the function will return a `CPU` device.

We can get a detailed information on the GPU cards installed on the compute node by calling `report_available_cuda_devices` function.

In [ ]:
report_available_cuda_devices()

In [ ]:
# instantiate a device using the only one available :P
device = get_device(True, 0)
device

### Create a BE-DICT (bystander) model 

**Note**: We use`bystander` and `haplotype` interchangeably in this demo.

In the sample data we are working with 👇, we have `Editor` column specifying what editor to use for the corresponding sequences. We also have `Outp_seq` column representing the observed *edited outcomes* of the input sequence with its corresponding probability (i.e. proportion of occurrence) of such outcome in `true_score` column.

In [8]:
for gr_name, gr_df in df.groupby(by=['Editor']):
    print(gr_name)
    display(gr_df)

ABE8e


,Editor,Inp_seq,Outp_seq,true_score
24,ABE8e,GTCTTCGACAAGAAGGGCAA,GTCTTCGACAAGAAGGGCAA,0.495025
25,ABE8e,GTCTTCGACAAGAAGGGCAA,GTCTTCGACAAGGAGGGCAA,0.000207
26,ABE8e,GTCTTCGACAAGAAGGGCAA,GTCTTCGACGAGAAGGGCAA,0.087479
27,ABE8e,GTCTTCGACAAGAAGGGCAA,GTCTTCGGCAAGAAGGGCAA,0.417289
28,ABE8e,CTGTGAAGCGCAGGGCAGGA,CTGTGAAGCGCAGGGCAGGA,0.698766
29,ABE8e,CTGTGAAGCGCAGGGCAGGA,CTGTGAAGCGCAGGGCAGGG,0.000305
30,ABE8e,CTGTGAAGCGCAGGGCAGGA,CTGTGAAGCGCAGGGCGGGA,0.000152
31,ABE8e,CTGTGAAGCGCAGGGCAGGA,CTGTGAAGCGCGGGGCAGGA,0.000457
32,ABE8e,CTGTGAAGCGCAGGGCAGGA,CTGTGAGGCGCAGGGCAGGA,0.011961
33,ABE8e,CTGTGAAGCGCAGGGCAGGA,CTGTGGAGCGCAGGGCAGGA,0.288359


ABEmax


,Editor,Inp_seq,Outp_seq,true_score
0,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGAATTTGTTGAGGGCGA,0.521655
1,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGAGTTTGTTGAGGGCGA,0.110202
2,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGGATTTGTTGAGGGCGA,0.180943
3,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGGGTTTGTTGAGGGCGA,0.168752
4,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGAATTTGTTGAGGGCGA,0.005775
5,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGAGTTTGTTGAGGGCGA,0.002406
6,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGGATTTGTTGAGGGCGA,0.001604
7,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGGGTTTGTTGAGGGCGA,0.004331
8,ABEmax,GAGAATGTCCCAGCCTGCCC,GAGAATGTCCCAGCCTGCCC,0.621697
9,ABEmax,GAGAATGTCCCAGCCTGCCC,GAGAGTGTCCCAGCCTGCCC,0.285483


BE4max


,Editor,Inp_seq,Outp_seq,true_score
12,BE4max,CGCCTGTGGGACGGGACACG,CGCCTGTGGGACGGGACACG,0.868093
13,BE4max,CGCCTGTGGGACGGGACACG,CGCTTGTGGGACGGGACACG,0.098624
14,BE4max,CGCCTGTGGGACGGGACACG,CGTCTGTGGGACGGGACACG,0.000747
15,BE4max,CGCCTGTGGGACGGGACACG,CGTTTGTGGGACGGGACACG,0.022776
16,BE4max,CGGCAACTACACCTGCGTCG,CGGCAACTACACCTGCGTCG,0.770898
17,BE4max,CGGCAACTACACCTGCGTCG,CGGCAACTATACCTGCGTCG,0.002473
18,BE4max,CGGCAACTACACCTGCGTCG,CGGCAATTACACCTGCGTCG,0.187112
19,BE4max,CGGCAACTACACCTGCGTCG,CGGCAATTATACCTGCGTCG,0.002958
20,BE4max,CGGCAACTACACCTGCGTCG,CGGTAACTACACCTGCGTCG,0.005479
21,BE4max,CGGCAACTACACCTGCGTCG,CGGTAACTATACCTGCGTCG,0.000048


Target-AID


,Editor,Inp_seq,Outp_seq,true_score
34,Target-AID,GTTCCATGTCGGGTGAACCT,GTTCCATGTCGGGTGAACCT,0.886470
35,Target-AID,GTTCCATGTCGGGTGAACCT,GTTCCATGTCGGGTGAATCT,0.000778
36,Target-AID,GTTCCATGTCGGGTGAACCT,GTTCCATGTTGGGTGAACCT,0.022551
37,Target-AID,GTTCCATGTCGGGTGAACCT,GTTCTATGTCGGGTGAACCT,0.010109
38,Target-AID,GTTCCATGTCGGGTGAACCT,GTTTCATGTCGGGTGAACCT,0.080093
39,Target-AID,TAACAGATGAGCTGGAAGTA,TAACAGATGAGCTGGAAGTA,0.856098
40,Target-AID,TAACAGATGAGCTGGAAGTA,TAACAGATGAGTTGGAAGTA,0.002439
41,Target-AID,TAACAGATGAGCTGGAAGTA,TAATAGATGAGCTGGAAGTA,0.141463


We can run the model for a specified `base editor` by following these steps:
- First instantiate `SeqProcessConfig` object which takes the following arguments:
    - protospacer sequence length (equal to `20` in this work)
    - start and end position of the protospacer sequence such as `(1, 20)` 
    - start and end position of editing window such as `(1, 20)`
    - numbering used to index the positions in the sequence above such as `1` based indexing (i.e. we start counting from 1). For example, we could use `0` based indexing or any indexing (including negative number) and in this case we provide the start and end of protospacer sequence and editing window based on the chosen indexing.
    

In [9]:
seqconfig_dataproc = SeqProcessConfig(20, (1,20), (1,20), 1)
print(seqconfig_dataproc)

+------------------------------------------------+-------+
|           Sequence processing Config           | Value |
+------------------------------------------------+-------+
|                sequence length                 |   20  |
|    sequence start index (0-based indexing)     |   0   |
|     sequence end index (0-based indexing)      |   19  |
| editable window start index (0-based indexing) |   0   |
|  editable window end index (0-based indexing)  |   19  |
|             offset start numbering             |   1   |
|              offset end numbering              |   20  |
+------------------------------------------------+-------+


Note that if we know the target editing window is in a smaller range than the whole sequence, we can specify that in the editing window start and end positions above. For example, in [Song et al. data](https://doi.org/10.1038/s41587-020-0573-5), editing window is from positions 3-10, hence we define our `SeqProcessConfig` object as follows: 
```python

seqconfig_dataproc = SeqProcessConfig(20, (1,20), (3,10), 1)

```

In fact sequences in the data frame above corresponding to `ABEmax` and `BE4max` are from Song et al. data and we will use this narrower edit range when running the models.

In [10]:
seqconfig_dataproc = SeqProcessConfig(20, (1,20), (3,10), 1)
print(seqconfig_dataproc)

+------------------------------------------------+-------+
|           Sequence processing Config           | Value |
+------------------------------------------------+-------+
|                sequence length                 |   20  |
|    sequence start index (0-based indexing)     |   0   |
|     sequence end index (0-based indexing)      |   19  |
| editable window start index (0-based indexing) |   2   |
|  editable window end index (0-based indexing)  |   9   |
|             offset start numbering             |   1   |
|              offset end numbering              |   20  |
+------------------------------------------------+-------+


As you can see above ☝️, independent of the user provided indexing (see `offset start numbering`), the `SeqProcessConfig` will generate `0`-based internal indexing :) 

In other words, the supplied indexing is a `user friendly` way to refer to positions in the sequence (i.e. edit window and start of protospacer sequence) without doing the conversion on your own.

   - Second we define `HaplotypeSeqProcessor` object which takes the following arguments:
      - target editor name as in {`ABEmax`, `BE4max`, `ABE8e`, `Target-AID`}
      - target canonical nucleotide and its expected conversion depending on the base editor used. For example, 
         - `(A,G)` for editors `ABEmax` and `ABE8e`
         - `(C,T)` for editors `BE4max` and `Target-AID`.
      - `SeqProcessConfig` instantiated above.

In [11]:
teditor = 'ABEmax'
cnv_nucl = ('A', 'G')
seq_processor = HaplotypeSeqProcessor(teditor, cnv_nucl, seqconfig_dataproc)

+-----------------------+--------+
|      Description      | Value  |
+-----------------------+--------+
|      Base editor      | ABEmax |
|   Target nucleotide   |   A    |
| Conversion nucleotide |   G    |
+-----------------------+--------+
+------------------------------------------------+-------+
|           Sequence processing Config           | Value |
+------------------------------------------------+-------+
|                sequence length                 |   20  |
|    sequence start index (0-based indexing)     |   0   |
|     sequence end index (0-based indexing)      |   19  |
| editable window start index (0-based indexing) |   2   |
|  editable window end index (0-based indexing)  |   9   |
|             offset start numbering             |   1   |
|              offset end numbering              |   20  |
+------------------------------------------------+-------+


   - Third we instantiate `SeqProcessConfig` similar to first step, to be used in generating data tensors
      - protospacer sequence length (equal to `20` in this work)
      - start and end position of the sequence such as `(1, 20)` 
      - start and end position of editing window such as `(1, 20)`
      - numbering used to index the positions in the sequence above such as `1` based indexing (i.e. we start counting from 1).

In [12]:
seqconfig_datatensgen = SeqProcessConfig(20, (1,20), (3 ,20), 1) 

The `SeqProcessConfig` used in the third step should mirror the object defined in first step except the `editing window` end position is always fixed to `20` (i.e. protospacer sequence length). We can change the `start` of `editing window` (i.e. using `3` instead of 1 given that we know the editing happens in the range `3-10`) or simply we specify equivalent start and end positions for both `protospacer sequence` and the `editing window` such as:

```python

seqconfig_datatensgen = SeqProcessConfig(20, (1,20), (1 ,20), 1) 

```

Now we reach to the last step where we define the model `BEDICT_HaplotypeModel` and supply the objects defined above:
   - `seq_processor` which is a `HaplotypeSeqProcessor` object defined in step 2.
   - `seqconfig_datatensgen` which is a `SeqProcessConfig` object defined in step 3.
   - `device` (i.e. CPU or GPU)

In [13]:
bedict_haplo = BEDICT_HaplotypeModel(seq_processor, seqconfig_datatensgen, device)

### Run a BE-DICT (bystander) model 

Given that we defined our model to run using `ABEmax` editor we will focus on sequences in our loaded data fame that have `Editor` column equal to `ABEmax`.

In [14]:
sample_df = df.loc[df['Editor'] == 'ABEmax'].copy()
sample_df

,Editor,Inp_seq,Outp_seq,true_score
0,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGAATTTGTTGAGGGCGA,0.521655
1,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGAGTTTGTTGAGGGCGA,0.110202
2,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGGATTTGTTGAGGGCGA,0.180943
3,ABEmax,ACAGAATTTGTTGAGGGCGA,ACAGGGTTTGTTGAGGGCGA,0.168752
4,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGAATTTGTTGAGGGCGA,0.005775
5,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGAGTTTGTTGAGGGCGA,0.002406
6,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGGATTTGTTGAGGGCGA,0.001604
7,ABEmax,ACAGAATTTGTTGAGGGCGA,ACGGGGTTTGTTGAGGGCGA,0.004331
8,ABEmax,GAGAATGTCCCAGCCTGCCC,GAGAATGTCCCAGCCTGCCC,0.621697
9,ABEmax,GAGAATGTCCCAGCCTGCCC,GAGAGTGTCCCAGCCTGCCC,0.285483


First, we prepare the data by running `prepare_data` method that takes the following args:
   - sample data frame such as `sample_df`
   - input sequence column name such as `Inp_seq`
   - output sequence column name (optional) such as `Outp_seq`
   - outcome column name (optional) representing probability of `output sequence`
   - batch size representing the number of sequences in the batch

#### Main (default) mode

The are different modes (i.e. options) to run the model. In the main (default) mode, we just provide the data frame that includes the input sequences and the model will generate the different outcome sequences and their corresponding probability.

In [15]:
dloader = bedict_haplo.prepare_data(sample_df,
                                    'Inp_seq',
                                    outpseq_col=None,
                                    outcome_col=None,
                                    renormalize=False,
                                    batch_size=500)

225it [00:00, 9589.56it/s]             
2it [00:00, 154.04it/s]
250it [00:00, 13083.00it/s]            
100%|██████████| 2/2 [00:00<00:00, 341.43it/s]

--- processing input data frame ---
number of NA: 0
--- checking for violating seqs ---
number of NA: 0
--- generating edit combinations ---
number of NA: 0
Generating tensors using sequence config:
 +------------------------------------------------+-------+
|           Sequence processing Config           | Value |
+------------------------------------------------+-------+
|                sequence length                 |   20  |
|    sequence start index (0-based indexing)     |   0   |
|     sequence end index (0-based indexing)      |   19  |
| editable window start index (0-based indexing) |   2   |
|  editable window end index (0-based indexing)  |   19  |
|             offset start numbering             |   1   |
|              offset end numbering              |   20  |
+------------------------------------------------+-------+
--- tensorizing ---
--- end ---
--- creating datatensor ---


After we parse the data, we can run the model to get predictions. We have trained `5` different models for every `base editor` (denoted by runs) which we will use on this data and take their average predictions. The `trained` bystander/haplotype models are found under the `trained_models/bystander` directory.
<pre>
trained_models
├── bystander
│   ├── ABE8e
│   │   └── train_val
│   │       ├── run_0
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_1
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_2
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_3
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       └── run_4
│   │           ├── config
│   │           │   ├── exp_options.pkl
│   │           │   └── mconfig.pkl
│   │           └── model_statedict
│   │               └── HaplotypeTransformer.pkl
│   ├── ABEmax
│   │   └── train_val
│   │       ├── run_0
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_1
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_2
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_3
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       └── run_4
│   │           ├── config
│   │           │   ├── exp_options.pkl
│   │           │   └── mconfig.pkl
│   │           └── model_statedict
│   │               └── HaplotypeTransformer.pkl
│   ├── BE4max
│   │   └── train_val
│   │       ├── run_0
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_1
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_2
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       ├── run_3
│   │       │   ├── config
│   │       │   │   ├── exp_options.pkl
│   │       │   │   └── mconfig.pkl
│   │       │   └── model_statedict
│   │       │       └── HaplotypeTransformer.pkl
│   │       └── run_4
│   │           ├── config
│   │           │   ├── exp_options.pkl
│   │           │   └── mconfig.pkl
│   │           └── model_statedict
│   │               └── HaplotypeTransformer.pkl
│   └── Target-AID
│       └── train_val
│           ├── run_0
│           │   ├── config
│           │   │   ├── exp_options.pkl
│           │   │   └── mconfig.pkl
│           │   └── model_statedict
│           │       └── HaplotypeTransformer.pkl
│           ├── run_1
│           │   ├── config
│           │   │   ├── exp_options.pkl
│           │   │   └── mconfig.pkl
│           │   └── model_statedict
│           │       └── HaplotypeTransformer.pkl
│           ├── run_2
│           │   ├── config
│           │   │   ├── exp_options.pkl
│           │   │   └── mconfig.pkl
│           │   └── model_statedict
│           │       └── HaplotypeTransformer.pkl
│           ├── run_3
│           │   ├── config
│           │   │   ├── exp_options.pkl
│           │   │   └── mconfig.pkl
│           │   └── model_statedict
│           │       └── HaplotypeTransformer.pkl
│           └── run_4
│               ├── config
│               │   ├── exp_options.pkl
│               │   └── mconfig.pkl
│               └── model_statedict
│                   └── HaplotypeTransformer.pkl
|
└── perbase
    ├── ABE8e
    │   └── train_val
    │       ├── run_0
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_1
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_2
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_3
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       └── run_4
    │           ├── config
    │           │   ├── exp_options.pkl
    │           │   └── mconfig.pkl
    │           └── model_statedict
    │               └── Transformer.pkl
    ├── ABEmax
    │   └── train_val
    │       ├── run_0
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_1
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_2
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_3
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       └── run_4
    │           ├── config
    │           │   ├── exp_options.pkl
    │           │   └── mconfig.pkl
    │           └── model_statedict
    │               └── Transformer.pkl
    ├── BE4max
    │   └── train_val
    │       ├── run_0
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_1
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_2
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       ├── run_3
    │       │   ├── config
    │       │   │   ├── exp_options.pkl
    │       │   │   └── mconfig.pkl
    │       │   └── model_statedict
    │       │       └── Transformer.pkl
    │       └── run_4
    │           ├── config
    │           │   ├── exp_options.pkl
    │           │   └── mconfig.pkl
    │           └── model_statedict
    │               └── Transformer.pkl
    └── Target-AID
        └── train_val
            ├── run_0
            │   ├── config
            │   │   ├── exp_options.pkl
            │   │   └── mconfig.pkl
            │   └── model_statedict
            │       └── Transformer.pkl
            ├── run_1
            │   ├── config
            │   │   ├── exp_options.pkl
            │   │   └── mconfig.pkl
            │   └── model_statedict
            │       └── Transformer.pkl
            ├── run_2
            │   ├── config
            │   │   ├── exp_options.pkl
            │   │   └── mconfig.pkl
            │   └── model_statedict
            │       └── Transformer.pkl
            ├── run_3
            │   ├── config
            │   │   ├── exp_options.pkl
            │   │   └── mconfig.pkl
            │   └── model_statedict
            │       └── Transformer.pkl
            └── run_4
                ├── config
                │   ├── exp_options.pkl
                │   └── mconfig.pkl
                └── model_statedict
                    └── Transformer.pkl
    
</pre>

We run the model to get prediction using `predict_from_dloader` method that takes the following args:
   - data loader such as `dloader` we generated above
   - path to trained model such as `crispr/trained_models/ABEmax/train_val/run_0`
   - outcome column name (optional) representing `true probability` of the output sequence. It is typically used if we need to keep the column in the generated prediction data frame to be later used for performance computation. 

Moreover, after we generate `5` predictions we can average them using `compute_avg_predictions` method that takes a concatenated data frame from the predictions of each run.

In [16]:
num_runs = 5 # number of models
pred_df_lst = [] # list to hold the prediction data frames for each model run
for run_num in range(num_runs):
    # specify model directory based on the chose editor
    model_dir = os.path.join(curr_pth, 'trained_models', 'bystander', teditor, 'train_val', f'run_{run_num}')
    print('run_num:', run_num)
    print('model_dir:', model_dir)

    pred_df = bedict_haplo.predict_from_dloader(dloader,
                                                model_dir, 
                                                outcome_col=None)
    pred_df['run_num'] = run_num
    pred_df_lst.append(pred_df)
    
# aggregate predictions in one data frame
pred_df_unif = pd.concat(pred_df_lst, axis=0, ignore_index=True)
# sanity check
check_na(pred_df_unif)

# compute average predictions
agg_df = bedict_haplo.compute_avg_predictions(pred_df_unif)
# sanity check
check_na(agg_df)


1it [00:00, 13.18it/s]
1it [00:00, 49.23it/s]
0it [00:00, ?it/s]

run_num: 0
model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_0
--- loading model config ---
--- building model ---
--- loading trained model ---
running prediction for base_editor: ABEmax | model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_0
run_num: 1
model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_1
--- loading model config ---
--- building model ---
--- loading trained model ---
running prediction for base_editor: ABEmax | model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_1
run_num: 2
model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_2
--- loading model config ---
--- building model ---
--- loading trained model ---
running prediction for base_editor: ABEmax | model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_2


1it [00:00, 60.83it/s]
1it [00:00, 42.39it/s]
1it [00:00, 50.75it/s]


run_num: 3
model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_3
--- loading model config ---
--- building model ---
--- loading trained model ---
running prediction for base_editor: ABEmax | model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_3
run_num: 4
model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_4
--- loading model config ---
--- building model ---
--- loading trained model ---
running prediction for base_editor: ABEmax | model_dir: /mnt/orisenbazuru/crispr/trained_models/bystander/ABEmax/train_val/run_4


In [17]:
agg_df

,Inp_seq,Outp_seq,pred_score
0,ACAGAATTTGTTGAGGGCGA,ACAGAATTTGTTGAGGGCGA,0.337371
1,ACAGAATTTGTTGAGGGCGA,ACAGAGTTTGTTGAGGGCGA,0.181658
2,ACAGAATTTGTTGAGGGCGA,ACAGGATTTGTTGAGGGCGA,0.197744
3,ACAGAATTTGTTGAGGGCGA,ACAGGGTTTGTTGAGGGCGA,0.139431
4,ACAGAATTTGTTGAGGGCGA,ACGGAATTTGTTGAGGGCGA,0.034776
5,ACAGAATTTGTTGAGGGCGA,ACGGAGTTTGTTGAGGGCGA,0.014884
6,ACAGAATTTGTTGAGGGCGA,ACGGGATTTGTTGAGGGCGA,0.016932
7,ACAGAATTTGTTGAGGGCGA,ACGGGGTTTGTTGAGGGCGA,0.007735
8,GAGAATGTCCCAGCCTGCCC,GAGAATGTCCCAGCCTGCCC,0.516905
9,GAGAATGTCCCAGCCTGCCC,GAGAGTGTCCCAGCCTGCCC,0.210303


### Visualize results

In [18]:
from criscas.data_preprocess import get_char
from IPython.core.display import HTML
from haplotype.data_preprocess import VizInpOutp_Haplotype

To visualize the results, we use `visualize_haplotype` method that takes the following args:
   - genereated predictions from the model (such as `agg_df`)
   - list of input sequences to visualize (we can pass all sequences such as `sample_df['Inp_seq']`)
   - input sequence column name in the generated `agg_df` (default is `Inp_seq`)
   - output sequence column name in the generated `agg_df`(default is `Outp_seq`)
   - prediction score column name in the generated `agg_df` (default is `prediction_score`)
   - prediction score threshold that filters out output sequences with probability below the specified threshold. If all output sequences are to be plotted then `predscore_thr` should be equal to `0.`.

In [19]:
tseqids = sample_df['Inp_seq']
res_html = bedict_haplo.visualize_haplotype(agg_df, 
                                            tseqids, 
                                            'Inp_seq', 
                                            'Outp_seq', 
                                            'pred_score', 
                                            predscore_thr=0.)
for seq_id in res_html:
    display(HTML(res_html[seq_id]))

250it [00:00, 14332.44it/s]            
100%|██████████| 2/2 [00:00<00:00, 123.64it/s]

number of NA: 0


Desc.,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Input sequence,G,A,G,A,A,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.5169,G,A,G,A,A,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.2103,G,A,G,A,G,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.1319,G,A,G,G,A,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.0631,G,A,G,G,G,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Position numbering,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Editable window (*),,,*,*,*,*,*,*,*,*,,,,,,,,,,
Sequence window (+),+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+


Desc.,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Input sequence,A,C,A,G,A,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.3374,A,C,A,G,A,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.1977,A,C,A,G,G,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.1817,A,C,A,G,A,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.1394,A,C,A,G,G,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0348,A,C,G,G,A,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0169,A,C,G,G,G,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0149,A,C,G,G,A,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0077,A,C,G,G,G,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Position numbering,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20


`res_html` is a dictionary that has `input sequences` as `keys` and the corresponding `value` is the generated `html table`.

We can also save the visualization of each sequence (or all sequences) in html format on disk using `HaplotypeVizFile` object.

In [20]:
from haplotype.data_preprocess import HaplotypeVizFile

`HaplotypeVizFile` object takes the path to `viz resources` needed to generate the html webpages. By default it is under `haplotype/viz_resources` folder.
```python
HaplotypeVizFile(os.path.join(curr_pth, 'haplotype', 'viz_resources'))
```

To create and save the files on disk, we use `create` method that takes three arguments:
   - generated `html table` from `visualize_haplotype` method
   - path where to save the file
   - name of the file (without extension)

In [21]:
vf = HaplotypeVizFile(os.path.join(curr_pth, 'haplotype', 'viz_resources'))
for seq_id in res_html:
    display(HTML(res_html[seq_id]))
    vf.create(res_html[seq_id], csv_dir, f'{teditor}_{seq_id}_haplotype')

Desc.,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Input sequence,G,A,G,A,A,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.5169,G,A,G,A,A,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.2103,G,A,G,A,G,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.1319,G,A,G,G,A,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Output sequence Prob.=0.0631,G,A,G,G,G,T,G,T,C,C,C,A,G,C,C,T,G,C,C,C
Position numbering,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Editable window (*),,,*,*,*,*,*,*,*,*,,,,,,,,,,
Sequence window (+),+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+,+


Desc.,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Input sequence,A,C,A,G,A,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.3374,A,C,A,G,A,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.1977,A,C,A,G,G,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.1817,A,C,A,G,A,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.1394,A,C,A,G,G,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0348,A,C,G,G,A,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0169,A,C,G,G,G,A,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0149,A,C,G,G,A,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Output sequence Prob.=0.0077,A,C,G,G,G,G,T,T,T,G,T,T,G,A,G,G,G,C,G,A
Position numbering,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20


We will find the results saved on disk under `sample_data` folder with the following structure

<pre>
sample_data
├── abemax_sampledata.csv
├── bystander_sampledata.csv
└── predictions_haplo
    ├── ABEmax_ACAGAATTTGTTGAGGGCGA_haplotype.html
    └── ABEmax_GAGAATGTCCCAGCCTGCCC_haplotype.html
</pre>